# Document search with embeddings

In [102]:
!pip install -U -q google.generativeai

In [103]:
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm

# Used to securely store your API key
from google.colab import userdata

from IPython.display import Markdown

In [104]:
# Or use `os.getenv('API_KEY')` to fetch an environment variable.
API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=API_KEY)

In [105]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001


### API changes to Embeddings with model embedding-001

For the new embeddings model, embedding-001, there is a new task type parameter and the optional title (only valid with task_type=`RETRIEVAL_DOCUMENT`).

These new parameters apply only to the newest embeddings models.The task types are:

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.
SEMANTIC_SIMILARITY	| Specifies the given text will be used for Semantic Textual Similarity (STS).
CLASSIFICATION	| Specifies that the embeddings will be used for classification.
CLUSTERING	| Specifies that the embeddings will be used for clustering.

Note: Specifying a `title` for `RETRIEVAL_DOCUMENT` provides better quality embeddings for retrieval.

In [106]:
title = "La prestazione occasionale 2017 e 2023"
sample_text = ("La prestazione occasionale 2017 e 2023")

model = 'models/embedding-001'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding)

{'embedding': [0.043930937, 0.0043483623, -0.031182509, 0.029410962, 0.03756037, 0.017048543, -0.004235978, -0.0050103664, 0.026933257, 0.031847537, -0.023051927, 0.010787278, -0.0073510413, 0.0155258365, -0.019124748, -0.06460495, -0.039103847, 0.02441746, -0.034809478, -0.017450778, -0.0032339108, 0.010542148, -0.0049116006, 0.0033414343, -0.0037959595, -0.0446708, 0.0021275855, -0.019789303, -0.008090899, -0.044343412, -0.087601036, 0.033638194, -0.04879754, 0.047524408, -0.022961015, -0.026465625, -0.0327514, 0.03789171, 0.002158931, 0.023253158, 0.011356481, -0.010650446, -0.028173247, -0.026068324, 0.0069446275, -0.025536332, 0.003977089, 0.036588445, 0.023488251, -0.08286156, 0.022676276, 0.010791082, 0.047499884, -0.012803996, 0.008266659, -0.050636314, 0.06268663, -0.048413265, -0.036869537, -0.0017487846, 0.0044489307, 0.016218655, 0.03375561, 0.03974302, -0.01911887, -0.016586801, -0.035882216, 0.0072563216, 0.006825989, -0.003988614, -0.0045524347, -0.019638585, 0.021208646

## Building an embeddings database



In [107]:
DOCUMENT1 = {
    "title": "Prestazione occasionale 2017",
    "content":
    """
    Prestazioni occasionali

Il Decreto Legge 24 aprile 2017, n. 50 Apre in una nuova scheda, convertito con modificazioni in Legge 21 giugno 2017, n. 96, disciplina le prestazioni occasionali (art. 54 bis).
In particolare, la normativa vigente ammette la possibilità di svolgere prestazioni di lavoro occasionale, intendendosi per tali le attività lavorative che danno luogo nel corso di un anno civile (1° gennaio – 31 dicembre di ciascun anno):  a) per ciascun prestatore, con riferimento alla totalità degli utilizzatori, a compensi di importo complessivamente non superiore a 5.000 euro;
 b) per ciascun utilizzatore, con riferimento alla totalità dei prestatori, a compensi di importo complessivamente non superiore a 5.000 euro;
 c) per le prestazioni complessivamente rese da ogni prestatore in favore del medesimo utilizzatore, a compensi di importo non superiore a 2.500 euro. c-bis) per ciascun prestatore, per le attività di “steward” negli impianti sportivi di cui Decreto del Ministro dell’Interno 8 agosto 2007, a compensi di importo complessivo non superiore a 5.000 euro.

Sono, tuttavia, computati in misura pari al 75% del loro importo i compensi per prestazioni di lavoro occasionale rese dai titolari di pensione di vecchiaia o di invalidità, dai giovani con meno di 25 anni di età, dai disoccupati (art. 19 del Decreto Legislativo 14 settembre 2015, n. 150), nonché dai percettori di prestazioni integrative del salario, di reddito di inclusione (REI), ovvero di altre prestazioni di sostegno del reddito.
Il prestatore ha diritto all’assicurazione per l’invalidità, la vecchiaia e i superstiti, con iscrizione alla Gestione separata dell’INPS, nonché all’assicurazione contro gli infortuni sul lavoro e le malattie professionali.
Inoltre, è previsto il diritto al riposo giornaliero, alle pause e ai riposi settimanali come regolati dal Decreto Legislativo 8 aprile 2003, n. 66 (articoli 7, 8 e 9), nonché l’estensione delle norme a tutela della salute e della sicurezza sul lavoro (art. 3, comma 8, del Decreto Legislativo 9 aprile 2008, n. 81).  I compensi percepiti dal prestatore sono esenti da imposizione fiscale, non incidono sullo stato di disoccupato e sono computabili ai fini della determinazione del reddito necessario per il rilascio o il rinnovo del permesso di soggiorno.
Il ricorso alle prestazioni di lavoro occasionali è, però, vietato rispetto a soggetti con i quali l’utilizzatore abbia in corso o abbia cessato da meno di 6 mesi un rapporto di lavoro subordinato o di collaborazione coordinata e continuativa.
L’art. 54 bis distingue, poi, le prestazioni occasionali rese mediante il sistema del “Libretto Famiglia” dai contratti di prestazione occasionale.
Per quanto attiene al Libretto Famiglia, l’utilizzatore può acquistare, attraverso la piattaforma informatica dedicata dell’INPS, un libretto nominativo prefinanziato per il pagamento delle prestazioni occasionali rese a suo favore da uno o più prestatori nell’ambito di: piccoli lavori domestici, compresi lavori di giardinaggio, di pulizia o di manutenzione; assistenza domiciliare ai bambini e alle persone anziane, ammalate o con disabilità; insegnamento privato supplementare; le attività svolte “steward” negli impianti sportivi, come da Decreto del Ministro dell’Interno 8 agosto 2007, limitatamente alle società sportive. Inoltre, mediante il Libretto Famiglia viene anche erogato il contributo per l’acquisto di servizi di baby-sitting, oppure per far fronte agli oneri della rete pubblica dei servizi per l’infanzia o dei servizi privati accreditati.
Ciascun Libretto Famiglia contiene titoli di pagamento, il cui valore nominale è pari a 10 euro, utilizzabili per compensare prestazioni di durata non superiore a un’ora. I contributi previdenziali e assicurativi, per ciascun titolo di pagamento erogato, sono interamente a carico dell’utilizzatore.
Il contratto di prestazione occasionale è il contratto mediante il quale un utilizzatore acquisisce con modalità semplificate prestazioni di lavoro occasionali o saltuarie di ridotta entità, entro i limiti di importo sopra indicati.
Possono fare ricorso a tale contratto: professionisti, lavoratori autonomi, imprenditori, associazioni, fondazioni ed altri enti di natura privata, nonché le amministrazioni pubbliche e le imprese agricole, sebbene con specifiche regolamentazioni.

È vietato il ricorso al contratto di prestazione occasionale:

    da parte degli utilizzatori che hanno alle proprie dipendenze più di 5 lavoratori subordinati a tempo indeterminato, ad eccezione delle aziende alberghiere e delle strutture ricettive che operano nel settore del turismo che hanno alle proprie dipendenze fino a 8 lavoratori per le attività lavorative rese dai pensionati, giovani con meno di 25 anni di età, disoccupati, percettori di prestazioni integrative del salario, di reddito di inclusione (REI), ovvero di altre prestazioni di sostegno del reddito.
    da parte delle imprese del settore agricolo, salvo che per le attività lavorative rese dai pensionati, giovani con meno di 25 anni di età, disoccupati, percettori di prestazioni integrative del salario, di reddito di inclusione (REI) ovvero di altre prestazioni di sostegno del reddito;
    da parte delle imprese dell’edilizia e di settori affini, delle imprese esercenti l’attività di escavazione o lavorazione di materiale lapideo, delle imprese del settore delle miniere, cave e torbiere;
    nell’ambito dell’esecuzione di appalti di opere o servizi.

Ai fini dell’attivazione del contratto di prestazione occasionale, ciascun utilizzatore deve versare attraverso la piattaforma informatica dell’INPS le somme utilizzabili per compensare le prestazioni.
Quanto al compenso, la misura minima oraria è pari a 9 euro, ad eccezione del settore agricolo per il quale il compenso minimo corrisponde all’importo della retribuzione oraria delle prestazioni di natura subordinata individuata dalla contrattazione collettiva.
Anche per il contratto di prestazione occasionale gli oneri contributivi previdenziali e assicurativi sono interamente a carico dell’utilizzatore.
Qualora non vengano rispettati il limite economico annuale di 2.500 euro in favore del medesimo utilizzatore oppure la durata complessiva della prestazione di 280 ore nell’arco dello stesso anno civile, salvo che per le pubbliche amministrazioni, il contratto si trasforma in un rapporto di lavoro a tempo pieno e indeterminato.
In caso di violazione dell’obbligo di comunicazione all’INPS oppure delle disposizioni che vietano il ricorso al contratto di prestazione occasionale è prevista una sanzione amministrativa pecuniaria da euro 500 a euro 2.500 per ogni prestazione lavorativa giornaliera per cui risulta accertata la violazione.
Si evidenzia, infine, che entro il 31 marzo di ogni anno il Ministero del Lavoro e delle Politiche Sociali, previo confronto con le parti sociali, trasmette alle Camere una relazione sullo sviluppo delle attività lavorative in questione.

In merito ai presupposti e alle modalità operative di fruizione del Libretto Famiglia e di ricorso al contratto di prestazione occasionale, si segnalano la circolare dell’INPS n. 107/2017File PDF - Apre in una nuova scheda (file pdf), nonché la circolare dell’INPS n. 103/2018File PDF - Apre in una nuova scheda (file pdf).
"""

}
DOCUMENT2 = {
    "title": "Prestazione occasionale 2023",
    "content":
    """
     Prestazioni occasionali: le novità 2023 della Legge di Bilancio
Con la Circolare n. 6/2023 l’Inps illustra le nuove disposizioni introdotte dall’art. 1, c. 342 e 343, della L. 29 dicembre 2022, n. 197
Di Laura Biarella
Avvocato in Perugia
Pubblicato il 01/02/2023

Con una Circolare 19 gennaio 2023, n. 6 (testo in calce) l’Inps ha fornito chiarimenti in materia di prestazioni occasionali, con riferimento alle nuove disposizioni introdotte dall’art. 1, c. 342 e 343, della Legge di Bilancio per il 2023.

Sommario

    La normativa
    Limiti economici per l’accesso al Libretto Famiglia e al Contratto di prestazione occasionale
    Nuovo limite dimensionale degli utilizzatori del Contratto di prestazione occasionale
    Divieto di utilizzo del Contratto di prestazione occasionale per l’agricoltura

La normativa

L’art. 54-bis del d.l. n. 50/2017, introdotto, in sede di conversione, dalla L. n. 96/2017, ha fornito una compiuta disciplina delle prestazioni di lavoro occasionale. Tale norma attribuisce all’INPS la gestione delle operazioni di registrazione degli utilizzatori e dei prestatori del lavoro occasionale, nonché delle relative comunicazioni dei rapporti di lavoro, da effettuarsi tramite un'apposita piattaforma informatica. È possibile per i datori di lavoro acquisire prestazioni di lavoro occasionali secondo due distinte modalità di utilizzo:

    il Libretto Famiglia,
    il Contratto di prestazione occasionale.

Possono utilizzare il Libretto Famiglia soltanto le persone fisiche, non nell’esercizio dell’attività professionale o d’impresa, per remunerare esclusivamente le prestazioni di lavoro occasionali rese in loro favore per:

    piccoli lavori domestici, inclusi i lavori di giardinaggio, di pulizia o di manutenzione;
    assistenza domiciliare ai bambini e alle persone anziane, ammalate o con disabilità;
    insegnamento privato supplementare.

Poi, l’art. 1, c. 368, della L. n. 205/2017, ha introdotto la possibilità dell’utilizzo del Libretto Famiglia per il pagamento, da parte delle società sportive di cui alla l. n. 91/81, delle prestazioni occasionali rese dagli steward per le attività di cui al decreto del Ministro dell'Interno 8 agosto 2007, in seguito sostituito dal decreto del Ministro dell’Interno 13 agosto 2019. Possono fare ricorso al Contratto di prestazione occasionale, nel rispetto dei vincoli indicati nell’art. 54-bis del d.l. n. 50/17:

    professionisti,
    lavoratori autonomi,
    imprenditori,
    associazioni,
    fondazioni e altri enti di natura privata,
    Amministrazioni pubbliche di cui all’art. 1, c. 2, d.lgs. n. 165/01.

Il c. 1 dell’art. 54-bis del d.l. n. 50/17 definisce le prestazioni di lavoro occasionali con riferimento a un limite di compenso annuo con il quale possono essere remunerate le attività lavorative che vengono rese nel rispetto delle previsioni del Libretto Famiglia e del Contratto di prestazione occasionale. Tale limite di compenso, riferito all’anno civile, è relativo a:

    ciascun prestatore, con riferimento alla totalità degli utilizzatori;
    ciascun utilizzatore, con riferimento alla totalità dei prestatori;
    prestazioni complessivamente rese da ogni prestatore in favore del medesimo utilizzatore.

Con l’art. 1, c. 342 e 343, della legge 29 di Bilancio per l’anno 2023 sono state introdotte alcune significative modificazioni alla disciplina di cui al citato art. 54-bis. Per tutti gli utilizzatori del Libretto Famiglia e del Contratto di prestazione occasionale è stato esteso l’importo massimo di compenso erogabile per anno civile dal singolo utilizzatore alla totalità dei prestatori che, dal 1° gennaio 2023, è pari a 10.000 euro.

È stata ampliata la platea di utilizzatori del Contratto di prestazione occasionale, consentendone il ricorso agli utilizzatori che hanno alle proprie dipendenze fino a 10 lavoratori subordinati a tempo indeterminato. Il legislatore ha altresì precisato che i nuovi limiti economici si applicano anche alle attività lavorative di natura occasionale svolte nell'ambito delle attività di discoteche, sale da ballo, night-club e simili. Risulta, infine, previsto un divieto generale di accesso al Contratto di prestazione occasionale da parte delle imprese operanti nel settore agricoltura.

Master online in diritto del lavoro

Su ShopAltalex è disponibile:
Master online in diritto del lavoro
di Falso Francesco, Scognamiglio Paolo, Staiano Rocchina, Pellecchia Roberto, 2024,
Acquista ora!

Limiti economici per l’accesso al Libretto Famiglia e al Contratto di prestazione occasionale

L’art. 1, c. 342, lettera a), della legge di Bilancio per il 2023 ha stabilito che il limite economico di cui all’art. 54-bis, c. 1, lettera b), del d.l. n. 50/17 (limite di compenso annuo con il quale possono essere remunerate le attività lavorative a titolo di prestazioni occasionali per ciascun utilizzatore con riferimento alla totalità dei prestatori) è pari a 10.000 euro.

Ne deriva che ciascun utilizzatore, con riferimento alla totalità dei prestatori, a decorrere dal 1° gennaio 2023 può erogare compensi di importo complessivamente non superiore a 10.000 euro per anno civile.

Restano fermi, invece, i limiti di compenso pari a 5.000 euro per ogni prestatore, con riferimento alla totalità degli utilizzatori (art. 54-bis, c. 1, lettera a), d.l. n. 50/17) e di 2.500 euro di compenso per le prestazioni complessivamente rese da ogni prestatore in favore del medesimo utilizzatore (art. 54 bis, c. 1, lettera c), d.l. n. 50/17).

Pertanto, i limiti di compenso annuo con il quale possono essere remunerate le attività lavorative a titolo di prestazioni occasionali, riferiti all’anno civile, sono pari a:

    per ciascun prestatore, con riferimento alla totalità degli utilizzatori, compensi di importo complessivamente non superiore a 5.000 euro;
    per ciascun utilizzatore, con riferimento alla totalità dei prestatori, compensi di importo complessivamente non superiore a 10.000 euro;
    per le prestazioni complessivamente rese da ogni prestatore in favore del medesimo utilizzatore, compensi di importo non superiore a 2.500 euro.

La novella normativa, introducendo il c. 1-bis all’art. 54-bis del d.l. n. 50/17, prevede che i suddetti limiti si applicano anche alle attività lavorative di natura occasionale, svolte nell'ambito delle attività di discoteche, sale da ballo, night-club e simili.
Nuovo limite dimensionale degli utilizzatori del Contratto di prestazione occasionale

In virtù delle modifiche apportate alla lettera a) del c. 14 dell’art. 54-bis del d.l. n. 50/17 dall’art. 1, c. 342, lettera d), punto 1), della legge di Bilancio 2023, è stato elevato a 10 lavoratori il previgente limite per il quale non era consentito l’accesso al Contratto di prestazione occasionale per gli utilizzatori con alle proprie dipendenze più di 5 lavoratori subordinati a tempo indeterminato.

Pertanto, dal 1° gennaio 2023, possono fare ricorso al Contratto di prestazione occasionale i datori di lavoro che hanno alle proprie dipendenze fino a dieci lavoratori subordinati a tempo indeterminato.

Il medesimo limite dimensionale di 10 lavoratori subordinati a tempo indeterminato si applica anche alle aziende alberghiere e alle strutture ricettive del settore turismo.
Divieto di utilizzo del Contratto di prestazione occasionale per l’agricoltura

L’art. 1, c. 342, della legge di Bilancio 2023 ha parzialmente abrogato, dal 1° gennaio 2023, quanto disposto dal c. 14, lettera b), dell’art. 54-bis del decreto-legge n. 50/2017, ed è stata inoltre prevista, dal c. 343 dell’art. 1 della legge di Bilancio 2023, l’abrogazione delle disposizioni che disciplinavano le prestazioni occasionali nel settore dell’agricoltura.

Pertanto, dal 1° gennaio 2023, è vietato l’utilizzo del Contratto di prestazione occasionale per le imprese operanti nel settore dell’agricoltura.
    """
}

documents = [DOCUMENT1, DOCUMENT2]

Organize the contents of the dictionary into a dataframe for better visualization.

In [108]:
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Prestazione occasionale 2017,\n Prestazioni occasionali\n\nIl Decreto Le...
1,Prestazione occasionale 2023,\n Prestazioni occasionali: le novità 2023...


Get the embeddings for each of these bodies of text. Add this information to the dataframe.

In [109]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,Prestazione occasionale 2017,\n Prestazioni occasionali\n\nIl Decreto Le...,"[0.017516384, -0.017656967, -0.0671682, 0.0452..."
1,Prestazione occasionale 2023,\n Prestazioni occasionali: le novità 2023...,"[0.030984657, -0.027299438, -0.058105864, 0.04..."


## Document search with Q&A

Now that the embeddings are generated, let's create a Q&A system to search these documents. You will ask a question about hyperparameter tuning, create an embedding of the question, and compare it against the collection of embeddings in the dataframe.

The embedding of the question will be a vector (list of float values), which will be compared against the vector of the documents using the dot product. This vector returned from the API is already normalized. The dot product represents the similarity in direction between two vectors.

The values of the dot product can range between -1 and 1, inclusive. If the dot product between two vectors is 1, then the vectors are in the same direction. If the dot product value is 0, then these vectors are orthogonal, or unrelated, to each other. Lastly, if the dot product is -1, then the vectors point in the opposite direction and are not similar to each other.

Note, with the new embeddings model (`embedding-001`), specify the task type as `QUERY` for user query and `DOCUMENT` when embedding a document text.

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.

In [110]:
query = "Quali sono i settori e le aziende che non possono utilizzare la prestazione occasionale nel 2023?"
model = 'models/embedding-001'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

Use the `find_best_passage` function to calculate the dot products, and then sort the dataframe from the largest to smallest dot product value to retrieve the relevant passage out of the database.

In [111]:
def find_best_passage(query, dataframe):
  """
Calcola le distanze tra la query e ciascun documento nel dataframe
  utilizzando il dot product.
  """
  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

View the most relevant document from the database:

In [112]:
passage = find_best_passage(query, df)
passage

"\n     Prestazioni occasionali: le novità 2023 della Legge di Bilancio\nCon la Circolare n. 6/2023 l’Inps illustra le nuove disposizioni introdotte dall’art. 1, c. 342 e 343, della L. 29 dicembre 2022, n. 197\nDi Laura Biarella\nAvvocato in Perugia\nPubblicato il 01/02/2023\n\nCon una Circolare 19 gennaio 2023, n. 6 (testo in calce) l’Inps ha fornito chiarimenti in materia di prestazioni occasionali, con riferimento alle nuove disposizioni introdotte dall’art. 1, c. 342 e 343, della Legge di Bilancio per il 2023.\n\nSommario\n\n    La normativa\n    Limiti economici per l’accesso al Libretto Famiglia e al Contratto di prestazione occasionale\n    Nuovo limite dimensionale degli utilizzatori del Contratto di prestazione occasionale\n    Divieto di utilizzo del Contratto di prestazione occasionale per l’agricoltura\n\nLa normativa\n\nL’art. 54-bis del d.l. n. 50/2017, introdotto, in sede di conversione, dalla L. n. 96/2017, ha fornito una compiuta disciplina delle prestazioni di lavoro 

## Question and Answering Application

Let's try to use the text generation API to create a Q & A system. Input your own custom data below to create a simple question and answering example. You will still use the dot product as a metric of similarity.

In [113]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""Sei un avvocato esperto in legge del lavoro che risponde alle domande utilizzando il testo del passaggio di riferimento incluso di seguito. \
  Assicurati di rispondere con una frase completa, che sia esauriente e che includa tutte le informazioni di base pertinenti. \
  Tuttavia, stai parlando a un pubblico non tecnico, quindi assicurati di scomporre concetti complicati e \
  assumere un tono amichevole e colloquiale. \
  Se il passaggio non è rilevante per la risposta, puoi ignorarlo.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [114]:
prompt = make_prompt(query, passage)
print(prompt)

Sei un avvocato esperto in legge del lavoro che risponde alle domande utilizzando il testo del passaggio di riferimento incluso di seguito.   Assicurati di rispondere con una frase completa, che sia esauriente e che includa tutte le informazioni di base pertinenti.   Tuttavia, stai parlando a un pubblico non tecnico, quindi assicurati di scomporre concetti complicati e   assumere un tono amichevole e colloquiale.   Se il passaggio non è rilevante per la risposta, puoi ignorarlo.
  QUESTION: 'Quali sono i settori e le aziende che non possono utilizzare la prestazione occasionale nel 2023?'
  PASSAGE: '      Prestazioni occasionali: le novità 2023 della Legge di Bilancio Con la Circolare n. 6/2023 l’Inps illustra le nuove disposizioni introdotte dall’art. 1, c. 342 e 343, della L. 29 dicembre 2022, n. 197 Di Laura Biarella Avvocato in Perugia Pubblicato il 01/02/2023  Con una Circolare 19 gennaio 2023, n. 6 (testo in calce) l’Inps ha fornito chiarimenti in materia di prestazioni occasion

Choose one of the Gemini content generation models in order to find the answer to your query.

In [115]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-pro
models/gemini-pro-vision


In [116]:
model = genai.GenerativeModel('gemini-pro')
answer = model.generate_content(prompt)

In [117]:
Markdown(answer.text)

Dal 1° gennaio 2023, le imprese operanti nel settore dell'agricoltura non possono utilizzare il Contratto di prestazione occasionale per assumere lavoratori.